In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

D:\Anaconda\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
data = {
    'ID':     [1,2,3,4,5,6],
    'Onion':  [1,0,0,1,1,1],
    'Potato':[1,1,0,1,1,1],
    'Burger': [1,1,0,0,1,1],
    'Milk':   [0,1,1,1,0,1],
    'Beer':   [0,0,1,0,1,0]
}

df = pd.DataFrame(data)
df = df[['ID','Onion','Potato','Burger','Milk','Beer']]


In [3]:
df

,ID,Onion,Potato,Burger,Milk,Beer
0,1,1,1,1,0,0
1,2,0,1,1,1,0
2,3,0,0,0,1,1
3,4,1,1,0,1,0
4,5,1,1,1,0,1
5,6,1,1,1,1,0


### 设置支持度（support）来选择频繁项集
    选择最小支持度为50%
    apriori(df, min_support=0.5, use_colname=True)

In [5]:
frequent_itemsets = apriori(df[['Onion','Potato','Burger','Milk','Beer']],min_support=0.50,use_colnames=True)

In [6]:
frequent_itemsets

,support,itemsets
0,0.666667,(Onion)
1,0.833333,(Potato)
2,0.666667,(Burger)
3,0.666667,(Milk)
4,0.666667,"(Onion, Potato)"
5,0.500000,"(Onion, Burger)"
6,0.666667,"(Potato, Burger)"
7,0.500000,"(Potato, Milk)"
8,0.500000,"(Onion, Potato, Burger)"


In [7]:
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1)

In [8]:
rules
#返回的是各个指标的数值，可以按照兴趣的指标排序观察，但具体解释还得参考实际数据的含义。

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Onion),(Potato),0.666667,0.833333,0.666667,1.00,1.200,0.111111,inf
1,(Potato),(Onion),0.833333,0.666667,0.666667,0.80,1.200,0.111111,1.666667
2,(Onion),(Burger),0.666667,0.666667,0.500000,0.75,1.125,0.055556,1.333333
3,(Burger),(Onion),0.666667,0.666667,0.500000,0.75,1.125,0.055556,1.333333
4,(Potato),(Burger),0.833333,0.666667,0.666667,0.80,1.200,0.111111,1.666667
5,(Burger),(Potato),0.666667,0.833333,0.666667,1.00,1.200,0.111111,inf
6,"(Onion, Potato)",(Burger),0.666667,0.666667,0.500000,0.75,1.125,0.055556,1.333333
7,"(Onion, Burger)",(Potato),0.500000,0.833333,0.500000,1.00,1.200,0.083333,inf
8,"(Potato, Burger)",(Onion),0.666667,0.666667,0.500000,0.75,1.125,0.055556,1.333333
9,(Onion),"(Potato, Burger)",0.666667,0.666667,0.500000,0.75,1.125,0.055556,1.333333


In [13]:
rules[(rules['lift']>1.125) & (rules['confidence']>0.8)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Onion),(Potato),0.666667,0.833333,0.666667,1.0,1.2,0.111111,inf
5,(Burger),(Potato),0.666667,0.833333,0.666667,1.0,1.2,0.111111,inf
7,"(Onion, Burger)",(Potato),0.500000,0.833333,0.500000,1.0,1.2,0.083333,inf


### 这几条结果就比较有价值了
 （洋葱和马铃薯）（汉堡和马铃薯）可以搭配着来卖
 如果洋葱和汉堡都在购物篮中，顾客买马铃薯的可能性就比较高


# 电影题材关联

In [15]:
movies = pd.read_csv('../data/movies.csv')
movies.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


数据中心包括电影名、电影类型标签，第一步还是先转换成one-hot格式

In [17]:
movies_ohe = movies.drop('genres',1).join(movies.genres.str.get_dummies())
pd.options.display.max_columns=100
movies_ohe.head()

,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
movies_ohe.shape

(9742, 22)

数据集包括9742部电影，一共有20种不同类型

In [20]:
movies_ohe.set_index(['movieId','title'], inplace=True)
movies_ohe.head()

,,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,title,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,Jumanji (1995),0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,Waiting to Exhale (1995),0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
5,Father of the Bride Part II (1995),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
frequent_itemsets_movies = apriori(movies_ohe,use_colnames=True,min_support=0.025)




In [22]:
frequent_itemsets_movies

,support,itemsets
0,0.187641,(Action)
1,0.129645,(Adventure)
2,0.062718,(Animation)
3,0.068158,(Children)
4,0.385547,(Comedy)
5,0.123075,(Crime)
6,0.045165,(Documentary)
7,0.447649,(Drama)
8,0.079963,(Fantasy)
9,0.100390,(Horror)


In [23]:
rules_movies = association_rules(frequent_itemsets_movies,metric='lift',min_threshold=1.25)

In [24]:
rules_movies

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Adventure),(Action),0.129645,0.187641,0.062615,0.482977,2.573940,0.038289,1.571224
1,(Action),(Adventure),0.187641,0.129645,0.062615,0.333698,2.573940,0.038289,1.306247
2,(Crime),(Action),0.123075,0.187641,0.042907,0.348624,1.857929,0.019813,1.247142
3,(Action),(Crime),0.187641,0.123075,0.042907,0.228665,1.857929,0.019813,1.136892
4,(Sci-Fi),(Action),0.100595,0.187641,0.046294,0.460204,2.452576,0.027419,1.504937
5,(Action),(Sci-Fi),0.187641,0.100595,0.046294,0.246718,2.452576,0.027419,1.193981
6,(Action),(Thriller),0.187641,0.194416,0.067235,0.358315,1.843034,0.030754,1.255420
7,(Thriller),(Action),0.194416,0.187641,0.067235,0.345829,1.843034,0.030754,1.241814
8,(Animation),(Adventure),0.062718,0.129645,0.025354,0.404255,3.118175,0.017223,1.460953
9,(Adventure),(Animation),0.129645,0.062718,0.025354,0.195566,3.118175,0.017223,1.165145


In [25]:
rules_movies[(rules_movies.lift)>4].sort_values(by=['lift'],ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
16,(Children),(Animation),0.068158,0.062718,0.031,0.454819,7.251799,0.026725,1.719213
17,(Animation),(Children),0.062718,0.068158,0.031,0.494272,7.251799,0.026725,1.842573


children和animation相关度比较高

In [26]:
movies[(movies.genres.str.contains('Children')) & (movies.genres.str.contains('Animation'))]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
12,13,Balto (1995),Adventure|Animation|Children
44,48,Pocahontas (1995),Animation|Children|Drama|Musical|Romance
205,239,"Goofy Movie, A (1995)",Animation|Children|Comedy|Romance
272,313,"Swan Princess, The (1994)",Animation|Children
...,...,...,...
9629,178827,Paddington 2 (2017),Adventure|Animation|Children|Comedy
9657,180987,Ferdinand (2017),Animation|Children|Comedy
9664,182293,Hare-um Scare-um (1939),Animation|Children|Comedy
9666,182299,Porky's Hare Hunt (1938),Animation|Children|Comedy
